In [5]:
from datetime import datetime, date, timedelta
import sys
import tkinter as tk
from tkinter import ttk, messagebox
import requests
import browser_cookie3
import pandas as pd
import PublicDataReader as pdr
from datetime import datetime, timedelta
import json
from urllib.parse import quote
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import requests
import io
import os
import time
import random
import webbrowser
import tkinter.filedialog
import datetime

# 새로 추가된 시계열 다운로드 관련 import
import shutil
import glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup # 맨 위 import 부분에 추가해 주세요.

# Windows 환경에서만 pywin32 import 및 사용
IS_WINDOWS = os.name == 'nt'
if IS_WINDOWS:
    try:
        import win32gui
        import win32con
        print("pywin32 라이브러리를 성공적으로 로드했습니다.")
    except ImportError:
        print("경고: pywin32 라이브러리가 설치되지 않았습니다. 'pip install pywin32'를 실행하여 설치하세요.")
        IS_WINDOWS = False

# Tkinter 고해상도(High DPI) 지원 시도
if IS_WINDOWS:
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)
        print("DPI 인식 설정을 시도했습니다.")
    except:
        print("DPI 인식 설정 중 오류가 발생했거나 지원되지 않는 환경입니다.")
        pass

# 프로그램 만료일 변수
PROGRAM_EXPIRATION_DATE_STR = "2025-08-31"

# 시계열 다운로드 설정 변수
TIMESERIES_DOWNLOAD_DIR = r"C:\Bi시각화\시계열"
KB_BASE_URL = "https://kbland.kr/webview.html#/main/statistics?channel=kbland"
KB_WEEKLY_TAB_XPATH = '//*[@id="__BVID__30___BV_tab_button__"]'
DOWNLOAD_BUTTON_XPATH = '//*[@id="reference2"]/div[1]/button'
INITIAL_LOAD_WAIT = 3
TAB_LOAD_WAIT = 3

# 매물수집기 관련 상수들
_BUILD_ID_ = "KVZ8_AwgDYT1YkrfeHfcs"
EXPIRATION_DATE = datetime.date(2025, 12, 31)

# API 엔드포인트
REGION_API_BASE_URL = f"https://fin.land.naver.com/_next/data/{_BUILD_ID_}/regions.json" 
GET_API_URL = "https://m.land.naver.com/cluster/ajax/articleList" 
GET_REGION_LIST_API_URL = "https://m.land.naver.com/map/getRegionList" 

# 매물수집기용 헤더 설정
NAVER_HEADERS = {
    "Host": "m.land.naver.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

class InvestmentTableProgram:
    def __init__(self, program_expiry_date_str):
        # ... (초기화 코드) ...
        # self.create_timeseries_analysis_page(...) 등 UI 생성 메서드 호출
        # 이 시점에 호출되는 모든 메서드는 이 __init__보다 위에 정의되어 있어야 합니다.
        # 즉, create_timeseries_analysis_page 가 여기서 호출되므로,
        # create_timeseries_analysis_page 보다 위에 download_kb_weekly_timeseries 같은
        # 필요한 메서드가 정의되어 있어야 합니다.
        
        self.root = tk.Tk()
        self.root.title("DAON_Searcher")

        # 프로그램 시작 시 사용 기한 점검
        self.check_usage_expiry(program_expiry_date_str) 

        # Matplotlib 폰트 설정
        try:
            font_name = fm.FontProperties(family='Malgun Gothic').get_name()
            plt.rcParams['font.family'] = font_name
            plt.rcParams['axes.unicode_minus'] = False
        except Exception as e:
            messagebox.showwarning("폰트 설정 오류", f"폰트 설정 중 오류 발생: {e}\n기본 폰트로 대체합니다.")
            plt.rcParams['font.family'] = 'DejaVu Sans'

        self.root.state('zoomed')

        # 페이지 구성 관리
        self.page_order = [1, 2, 3, 4, 5]
        self.page_names = {
            1: "투자테이블", 2: "시세차트", 3: "시계열분석", 4: "매물수집기", 5: "페이지5"
        }
        self.current_page = 1 
        self.current_page_frame = None

        # 필터 설정 변수들
        self.region_var = tk.StringVar(value="4481000000") 
        self.sido_var = tk.StringVar()                     
        self.sigungu_var = tk.StringVar()                  
        self.region_data = {}                              

        # 날짜, 면적, 증감율 등 기타 필터 변수
        self.start_date_var = tk.StringVar()
        self.end_date_var = tk.StringVar()
        end_date = datetime.now()
        start_date = end_date - pd.Timedelta(days=180)
        self.start_date_var.set(start_date.strftime("%Y-%m-%d"))
        self.end_date_var.set(end_date.strftime("%Y-%m-%d"))
        self.exclusive_area_min = tk.StringVar(value="0")
        self.exclusive_area_max = tk.StringVar(value="999")
        self.max_years = tk.StringVar(value="30")
        self.max_unit_price = tk.StringVar(value="999999")
        self.rate_min = tk.StringVar(value="-100")
        self.rate_max = tk.StringVar(value="100")

        # 데이터 저장 변수
        self.raw_data = None
        self.filtered_data = None
        self.selected_apartments = {}

        # 시계열 분석 관련 변수 추가
        self.timeseries_download_path_var = tk.StringVar()
        self.set_default_timeseries_download_path()
        self.timeseries_sido_var = tk.StringVar()
        self.timeseries_sigungu_var = tk.StringVar()
        self.timeseries_period_start_var = tk.StringVar()
        self.timeseries_period_end_var = tk.StringVar()
        
        # 차트 데이터 선택 변수들 (새로 추가)
        self.chart_data_vars = {
            '매수심리': tk.BooleanVar(value=True),  # 초기에 선택됨
            '전세현황': tk.BooleanVar(value=False),
            '전세수급': tk.BooleanVar(value=False),
            '매매전망': tk.BooleanVar(value=False),
            '전세전망': tk.BooleanVar(value=False),
            '매매지수': tk.BooleanVar(value=False),
            '전세지수': tk.BooleanVar(value=False),
            '매매증감': tk.BooleanVar(value=False),
            '전세증감': tk.BooleanVar(value=False)
        }
        
        # 기본 기간 설정 (최근 1년)
        end_date_ts = datetime.now()
        start_date_ts = end_date_ts - pd.Timedelta(days=365)
        self.timeseries_period_start_var.set(start_date_ts.strftime("%Y-%m-%d"))
        self.timeseries_period_end_var.set(end_date_ts.strftime("%Y-%m-%d"))
        
        # 시계열 차트 관련 변수
        self.timeseries_chart_frame = None
        self.timeseries_fig = None
        self.timeseries_ax = None
        self.timeseries_canvas = None
        self.current_timeseries_data = None

        # 엑셀 내보내기 경로 저장 변수 추가
        self.export_path_var = tk.StringVar()
        self.set_default_export_path()

        # 차트 관련 변수
        self.fig = None; self.ax = None; self.ax2 = None; self.canvas = None
        self.chart_info_labels = {}; self.current_chart_data = None
        self.current_selected_apt_info = None

        # 차트 옵션 변수
        self.show_sise_sale_var = tk.BooleanVar(value=True)
        self.show_sise_lease_var = tk.BooleanVar(value=True)
        self.show_sise_ratio_var = tk.BooleanVar(value=True)
        self.show_real_sale_var = tk.BooleanVar(value=True)
        self.show_real_lease_var = tk.BooleanVar(value=True)
        self.chart_start_date_var = tk.StringVar()
        self.chart_end_date_var = tk.StringVar()

        # 실거래 리스트용 Treeviews
        self.sale_real_tran_tree = None
        self.lease_real_tran_tree = None

        # 매물수집기 관련 변수들
        self._stop_flag = False
        self._regions_data = {} 
        self.fetched_article_data = []

        # 지역코드.txt 로드 메서드 호출
        self.load_region_data() 

        # UI 레이아웃 핵심
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 내비게이션 바
        self.create_navigation_bar(self.main_frame) 

        # 페이지 내용이 표시될 컨텐츠 프레임
        self.content_frame = ttk.Frame(self.main_frame)
        self.content_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        # 로딩 레이블
        self.loading_label = ttk.Label(self.main_frame, text="프로그램 준비 완료", font=('Arial', 12))
        self.loading_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

        # 페이지 프레임 생성 및 UI 그리기 호출
        self.page_frames = {} 
        
        # 1페이지 (투자테이블) 프레임 생성 및 UI 그리기
        self.page_frames[1] = ttk.Frame(self.content_frame)
        self.create_investment_table_page(self.page_frames[1])

        # 2페이지 (시세차트) 프레임 생성 및 UI 그리기 - 매물 검색 기능 제거됨
        self.page_frames[2] = ttk.Frame(self.content_frame)
        self.create_chart_page(self.page_frames[2])

        # 3페이지 (시계열분석) 프레임 생성 및 UI 그리기
        self.page_frames[3] = ttk.Frame(self.content_frame)
        self.create_timeseries_analysis_page(self.page_frames[3])

        # 4페이지 (매물수집기) - 새로 추가
        self.page_frames[4] = ttk.Frame(self.content_frame)
        self.create_property_collector_page(self.page_frames[4])
        
        # 5페이지는 임시 페이지
        self.page_frames[5] = ttk.Frame(self.content_frame)
        ttk.Label(self.page_frames[5], text="5페이지 입니다. (임시)", font=('Arial', 20)).pack(pady=50)

        # 시계열 다운로드 폴더 생성 확인
        self.ensure_timeseries_download_dir()

        # 프로그램 시작 시 초기 페이지 로드
        self.load_page(1)

    # User-Agent 기본 헤더 설정
    DEFAULT_HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://new.land.naver.com/",
        "Origin": "https://new.land.naver.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site"
    }

    def check_usage_expiry(self, expiry_date_str):
        """프로그램 사용 기한 점검"""
        try:
            expiration_date = datetime.strptime(expiry_date_str, "%Y-%m-%d").date()
        except ValueError:
            messagebox.showwarning("기한 설정 오류", f"제공된 만료일 형식이 올바르지 않습니다: {expiry_date_str}. (YYYY-MM-DD 형식이어야 합니다.)")
            return

        today = date.today()
        days_left = (expiration_date - today).days

        if today > expiration_date:
            messagebox.showwarning(
                "사용 기한 만료",
                f"프로그램 사용 기한({expiration_date})이 만료되었습니다.\n지속적인 사용을 위해 관리자 승인이 필요합니다.",
                parent=self.root
            )
            self.root.destroy() 
            sys.exit() 
            return

        elif 0 <= days_left <= 10:
            messagebox.showwarning(
                "사용 기한 임박",
                f"프로그램 사용 기한이 {days_left}일 남았습니다 ({expiration_date}).\n지속적인 사용을 위해 관리자 승인이 필요합니다.",
                parent=self.root
            )

    def set_default_timeseries_download_path(self):
        """시계열 다운로드 기본 경로 설정"""
        self.timeseries_download_path_var.set(TIMESERIES_DOWNLOAD_DIR)

    def ensure_timeseries_download_dir(self):
        """시계열 다운로드 폴더 생성 확인"""
        download_path = self.timeseries_download_path_var.get()
        if not os.path.exists(download_path):
            try:
                os.makedirs(download_path)
                print(f"시계열 다운로드 폴더를 생성했습니다: {download_path}")
            except OSError as e:
                messagebox.showerror("오류", f"시계열 다운로드 폴더 생성 실패: {e}")

    def show_error(self, error_msg):
        """일반 오류 메시지를 표시하는 범용 메서드"""
        self.loading_label.config(text="오류가 발생했습니다.")
        messagebox.showerror("오류", error_msg)        

    def fetch_naver_api_data(self, url):
        """
        네이버 부동산 API에서 데이터를 가져오는 범용 메서드
        - 브라우저에서 복사한 쿠키를 하드코딩
        - browser_cookie3 없이 동작
        """
        try:
            headers = {
                'accept': 'application/json, text/plain, */*',
                'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                'referer': 'https://fin.land.naver.com/',
                'user-agent': (
                    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/138.0.0.0 Safari/537.36'
                ),
                # ★ 여기 쿠키 값은 크롬 개발자도구에서 복사해서 붙여넣으세요
                'cookie': 'NAC=LcV5BogfnITfC; NNB=53JOHN6SM5RGQ; SHOW_FIN_BADGE=Y; bnb_tooltip_shown_finance_v1=true; _fwb=220FhMGmpQ2BtTNICGStK6Z.1752616946035; landHomeFlashUseYn=Y; ASID=7a2b3579000001985a70dc080000001b; nhn.realestate.article.rlet_type_cd=A01; nhn.realestate.article.trade_type_cd=\"\"; realestate.beta.lastclick.cortar=4400000000; PROP_TEST_KEY=1754632854081.db0b658403da2bf0f24b506613cf8d9acf878c7af22296020db8166f2844b2db; PROP_TEST_ID=44995f4422776c30b611ccf580060a39260c013b9d8179a5129d683c639425e5; page_uid=j5kX6dqo1LVssM9bZP8ssssssJ0-281326; NACT=1; SRT30=1755142689; SRT5=1755142689; BUC=G3KGLDoPM5S83hxSi_0iws26uo9I8bUF-RVCvmS3OMo=',
            }

            session = requests.Session()
            session.headers.update(headers)

            # 1~3초 랜덤 대기 (429 방지)
            time.sleep(random.uniform(5, 10))

            response = session.get(url, timeout=10)
            response.raise_for_status()

            if response.headers.get('content-type', '').startswith('application/json'):
                return response.json()
            else:
                print(f"경고: JSON 응답이 아님. Content-Type: {response.headers.get('content-type')}")
                print(response.text[:500])
                return None

        except requests.exceptions.RequestException as e:
            print(f"API 요청 오류: {e}")
            if e.response is not None:
                print(f"응답 코드: {e.response.status_code}")
                print(e.response.text[:500])
            return None
        except Exception as e:
            print(f"예상치 못한 오류 발생: {e}")
            return None
            
    def set_default_export_path(self):
        """Excel 내보내기 기본 경로 설정"""
        home_dir = os.path.expanduser("~")
        download_dir = os.path.join(home_dir, "Downloads")
        self.export_path_var.set(download_dir)

    def load_region_data(self):
        """지역 코드 파일 로드 및 파싱 (콤보박스 등에 필요)"""
        # GitHub Raw URL
        github_raw_url = "https://raw.githubusercontent.com/kaga-fo/DaonSearcher_Project/refs/heads/main/%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C.txt"
        
        # 로컬 파일 경로
        local_fallback_path = r"C:\Users\kagaj\code\지역코드.txt"
        
        self.region_data = {}

        file_source_name = ""
        file_content_object = None

        try:
            # 1. GitHub에서 로드 시도
            file_source_name = "GitHub"
            print(f"[{file_source_name}]에서 지역코드 파일 로드 시도: {github_raw_url}")
            response = requests.get(github_raw_url, timeout=10)
            response.raise_for_status()

            file_content_object = io.StringIO(response.text)
            print(f"[{file_source_name}]에서 지역코드 파일 성공적으로 로드.")

        except requests.exceptions.RequestException as e:
            # 2. GitHub 로드 실패 시 로컬 파일 시도
            print(f"[{file_source_name}]에서 지역코드 파일 로드 중 오류 발생: {e}. 로컬 파일로 대체 시도.")
            if os.path.exists(local_fallback_path):
                file_source_name = "로컬 파일"
                try:
                    file_content_object = open(local_fallback_path, 'r', encoding='utf-8-sig') 
                    print(f"[{file_source_name}] '{local_fallback_path}' 성공적으로 로드.")
                except Exception as ex:
                    messagebox.showerror("오류", f"로컬 지역코드 파일 로드 중 오류 발생: {str(ex)}\n파일 인코딩 또는 경로를 확인해주세요.")
                    self.region_data = {}
                    return
            else:
                messagebox.showerror("오류", f"지역코드 파일이 없습니다.\nGitHub에서도 로컬({local_fallback_path})에서도 찾을 수 없습니다.")
                self.region_data = {}
                return
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 처리 중 알 수 없는 오류 발생: {str(e)}")
            self.region_data = {}
            return

        # 파싱 로직
        try:
            for line in file_content_object:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                
                if len(parts) >= 3: 
                    code = parts[0]
                    sido = parts[1]
                    sigungu = parts[2]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][sigungu] = code
                elif len(parts) == 2: 
                    code = parts[0]
                    sido = parts[1]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][''] = code
                else:
                    print(f"경고: 지역코드 파일 '{file_source_name}'의 형식이 올바르지 않습니다 (최소 2개 항목 필요): {line}")

            print(f"지역코드 파싱 완료. 총 {len(self.region_data)}개의 시도 데이터.")
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 파싱 중 오류 발생: {str(e)}")
            self.region_data = {}
        finally:
            if hasattr(file_content_object, 'close'): 
                file_content_object.close()

    # 매물수집기 관련 메서드들
    
    # 부동산 유형 코드 매핑 (한글 이름 -> API 코드)
    _RLET_TYPE_CODES = {
        '아파트': 'APT',
        '빌라': 'VL:YR:DSD',  # 빌라 선택 시 빌라, 연립, 다세대 모두 포함
        '오피스텔': 'OPST',
    }

    # 거래 유형 코드 매핑 (한글 이름 -> API 코드)
    _TRAD_TYPE_CODES = {
        '매매': 'A1',
        '전세': 'B1',
        '월세': 'B2',
    }

    # 평형대 코드 및 spcMin/spcMax 매핑 (사용자 표시명 -> {spcMin, spcMax})
    _PYEONG_TYPE_RANGES = {
        '~10평': {'spcMax': 33},
        '10평~20평': {'spcMin': 33, 'spcMax': 66},
        '20평~30평': {'spcMin': 66, 'spcMax': 99},
        '30평~40평': {'spcMin': 99, 'spcMax': 132},
        '40평~50평': {'spcMin': 132, 'spcMax': 165},
        '50평~60평': {'spcMin': 165, 'spcMax': 198},
        '60평~70평': {'spcMin': 198, 'spcMax': 231},
        '70평~': {'spcMin': 231, 'spcMax': 900000000}, # 매우 큰 값으로 무제한 의미
    }

    # 줌 레벨에 따른 대략적인 델타 값 (시행착오를 통해 조정 필요)
    _Z_LEVEL_DELTAS = {
        10: {'lat_delta': 0.2, 'lon_delta': 0.2},
        11: {'lat_delta': 0.1, 'lon_delta': 0.1},
        12: {'lat_delta': 0.05, 'lon_delta': 0.05}, 
        13: {'lat_delta': 0.025, 'lon_delta': 0.025},
        14: {'lat_delta': 0.012, 'lon_delta': 0.012},
        15: {'lat_delta': 0.006, 'lon_delta': 0.006},
        16: {'lat_delta': 0.003, 'lon_delta': 0.003},
    }

    def update_status(self, message):
        """상태 메시지 업데이트"""
        if hasattr(self, 'status_label'):
            self.status_label.config(text=message)
            self.root.update_idletasks()
        else:
            self.loading_label.config(text=message)
            self.root.update_idletasks()

    def _safe_request(self, method, url, headers, params=None, json_data=None, timeout=10, api_name="API"):
        try:
            print(f"\n--- API 요청 시작: {api_name} ---")
            print(f"  URL: {url}")
            print(f"  Method: {method.upper()}")
            print(f"  Params: {params}")
            
            if method.lower() == 'post':
                response = requests.post(url, headers=headers, json=json_data, timeout=timeout)
            elif method.lower() == 'get':
                response = requests.get(url, headers=headers, params=params, timeout=timeout)
            else:
                raise ValueError("지원하지 않는 HTTP 메서드입니다.")

            response.raise_for_status() 

            print(f"  Status Code: {response.status_code}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                delay_time = random.uniform(5, 15)
                self.update_status(f"⚠️ {api_name} 429 오류 발생. {delay_time:.2f}초 후 재시도.")
                time.sleep(delay_time)
                return "RETRY"
            else:
                response_text = e.response.text if hasattr(e.response, 'text') else '응답 본문 없음'
                self.root.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("HTTP 오류", f"{api_name} 요청 중 오류 발생:\n상태코드: {e_val.response.status_code}\nURL: {e_val.request.url}\n응답: {res_text_val}"))
                self.update_status(f"⛔️ {api_name} HTTP 오류: {e.response.status_code}")
        except requests.exceptions.ConnectionError as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("연결 오류", f"{api_name} 서버 연결 실패:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 연결 오류: {e}")
        except requests.exceptions.Timeout as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("시간 초과", f"{api_name} 요청 시간 초과:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 시간 초과: {e}")
        except requests.exceptions.RequestException as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("요청 오류", f"{api_name} 요청 중 알 수 없는 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 알 수 없는 오류: {e}")
        except json.JSONDecodeError as e:
            response_text = response.text if 'response' in locals() else '없음'
            self.root.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("JSON 파싱 오류", f"{api_name} 응답 JSON 디코딩 실패:\n{e_val}\n응답 본문: {res_text_val}"))
            self.update_status(f"⛔️ {api_name} JSON 파싱 오류: {e}")
        except Exception as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("알 수 없는 오류", f"{api_name} 예상치 못한 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 예상치 못한 오류: {e}")
        return None

    # 다운로드 관련 메서드들

    def wait_for_download_completion(self, download_path, timeout=60):
        """다운로드 완료 대기 유틸리티 메서드"""
        print(f"다운로드 폴더 '{download_path}'에서 새로운 파일 다운로드를 기다립니다... (최대 {timeout}초)")
        seconds = 0
        last_size = -1
        file_path = None
        start_time = time.time()

        existing_files = {os.path.join(download_path, f): os.path.getctime(os.path.join(download_path, f))
                          for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))}

        while time.time() - start_time < timeout:
            time.sleep(1)
            seconds += 1

            list_of_files = [os.path.join(download_path, f) for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))]

            if not list_of_files:
                 continue

            new_files = [(f, os.path.getctime(f)) for f in list_of_files if f not in existing_files]

            if new_files:
                latest_new_file, latest_ctime = max(new_files, key=lambda item: item[1])

                if not latest_new_file.endswith('.crdownload'):
                     file_path = latest_new_file
                     current_size = os.path.getsize(file_path)

                     if current_size > 0 and current_size == last_size and seconds > 2:
                         print(f"파일 크기 안정됨: {current_size} bytes. 다운로드 완료 추정.")
                         return file_path
                     else:
                         last_size = current_size

        print(f"다운로드 완료 대기 시간 초과 ({timeout}초). 새로운 파일이 감지되지 않았거나 다운로드가 완료되지 않았습니다.")
        return None

    def process_downloaded_file(self, downloaded_file_path, target_name):
        """다운로드된 파일을 복사하고 원하는 이름으로 변경하여 저장"""
        if not downloaded_file_path or not os.path.exists(downloaded_file_path):
            print("다운로드된 파일을 찾을 수 없습니다.")
            return False

        download_path = self.timeseries_download_path_var.get()
        target_file_name = f"{target_name}.xlsx"
        target_file_path = os.path.join(download_path, target_file_name)

        print(f"다운로드된 원본 파일: {os.path.basename(downloaded_file_path)}")
        print(f"복사본 대상 파일명: {target_file_name}")

        # 기존에 같은 이름의 대상 파일이 있으면 삭제
        if os.path.exists(target_file_path):
            print(f"기존 대상 파일 삭제 시도: {target_file_name}")
            try:
                os.remove(target_file_path)
                print("기존 파일 삭제 완료.")
            except OSError as e:
                print(f"기존 대상 파일 삭제 실패: {e}")
                messagebox.showerror("오류", f"기존 파일 삭제 실패: {target_file_name}\n{e}\n파일이 사용 중인지 확인해 보세요.")
                return False

        # 다운로드된 파일을 원하는 이름으로 복사
        try:
            shutil.copy2(downloaded_file_path, target_file_path)
            print(f"파일 복사 완료: {os.path.basename(downloaded_file_path)} -> {target_file_name}")
            return True
        except OSError as e:
            print(f"파일 복사 실패: {e}")
            messagebox.showerror("오류", f"파일 복사 실패: {os.path.basename(downloaded_file_path)} -> {target_file_name}\n{e}")
            return False

    def download_kb_weekly_timeseries(self):
        """KB 주간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 주간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_weekly_download)
        thread.daemon = True
        thread.start()

    def download_kb_monthly_timeseries(self):
        """KB 월간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 월간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_monthly_download)
        thread.daemon = True
        thread.start()

    def _run_kb_weekly_download(self):
        """KB 주간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None
        
        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 주간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # '주간시계열' 탭 클릭
            try:
                print(f"주간시계열 탭 찾기 및 클릭 시도: {KB_WEEKLY_TAB_XPATH}")
                weekly_tab = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, KB_WEEKLY_TAB_XPATH))
                )
                weekly_tab.click()
                print("주간시계열 탭 클릭 완료.")
                print(f"탭 내용 로딩 및 다운로드 버튼 활성화 대기 ({TAB_LOAD_WAIT}초)...")
                time.sleep(TAB_LOAD_WAIT)

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"주간시계열 탭을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"주간시계열 탭 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 다운로드 버튼 클릭
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")
            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "주간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 주간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 주간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 주간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 주간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 주간 시계열 다운로드 프로세스 종료.")

    def _run_kb_monthly_download(self):
        """KB 월간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None

        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 월간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # 다운로드 버튼 클릭 (월간시계열은 기본 탭이므로 탭 클릭 불필요)
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "월간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 월간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 월간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 월간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 월간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 월간 시계열 다운로드 프로세스 종료.")

    # UI 생성 메서드들

    def create_navigation_bar(self, parent):
        """메인 내비게이션 바 생성"""
        nav_frame = ttk.Frame(parent)
        nav_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Button(nav_frame, text="페이지 순서 변경", command=self.change_page_order).pack(side=tk.LEFT, padx=(0, 20))

        self.page_buttons = {}
        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def change_page_order(self):
        """페이지 순서 변경 (임시 구현)"""
        messagebox.showinfo("정보", "페이지 순서 변경 기능은 향후 구현 예정입니다.")

    def load_page(self, page_num):
        """페이지 로드"""
        if self.current_page_frame:
            self.current_page_frame.pack_forget()
        
        self.current_page = page_num
        self.current_page_frame = self.page_frames[page_num]
        self.current_page_frame.pack(fill=tk.BOTH, expand=True)
        
        # 버튼 상태 업데이트
        for p, btn in self.page_buttons.items():
            if p == page_num:
                btn.config(state=tk.DISABLED)
            else:
                btn.config(state=tk.NORMAL)

    def create_investment_table_page(self, parent_frame):
        """1페이지: 투자테이블 UI 생성"""
        title_label = ttk.Label(parent_frame, text="투자테이블", font=('Arial', 16, 'bold')) 
        title_label.pack(pady=(0, 20))

        filter_frame = ttk.LabelFrame(parent_frame, text="필터 설정", padding=10) 
        filter_frame.pack(fill=tk.X, pady=(0, 10))

        self.create_filter_ui(filter_frame)

        result_frame = ttk.LabelFrame(parent_frame, text="조회 결과", padding=10)
        result_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        self.create_result_table(result_frame)

    def create_filter_ui(self, parent):
        """투자테이블 필터 UI 생성"""
        row = 0
        padx_val = 3
        pady_val = 3

        # 컬럼 가중치 설정 (8개 열)
        parent.grid_columnconfigure(0, weight=0, minsize=40)
        parent.grid_columnconfigure(1, weight=1)             
        parent.grid_columnconfigure(2, weight=0, minsize=40)
        parent.grid_columnconfigure(3, weight=1)             
        parent.grid_columnconfigure(4, weight=0, minsize=20)
        parent.grid_columnconfigure(5, weight=1)
        parent.grid_columnconfigure(6, weight=1)
        parent.grid_columnconfigure(7, weight=0, minsize=40)

        # Row 0: 시도, 시군구
        ttk.Label(parent, text="시도:").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox = ttk.Combobox(parent, textvariable=self.sido_var, values=list(self.region_data.keys()), state="readonly", width=15)
        self.sido_combobox.grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected)

        ttk.Label(parent, text="시군구:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sigungu_combobox = ttk.Combobox(parent, textvariable=self.sigungu_var, state="readonly", width=15)
        self.sigungu_combobox.grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)

        if self.region_data:
            first_sido = list(self.region_data.keys())[0]
            self.sido_var.set(first_sido)
            self.on_sido_selected(None)
        else:
            self.sido_combobox.config(state="disabled")
            self.sigungu_combobox.config(state="disabled")
            messagebox.showwarning("경고", "지역코드 파일 로드에 실패하여 지역 선택 기능을 사용할 수 없습니다.")
        row += 1 

        # Row 1: 시작일, 종료일
        ttk.Label(parent, text="시작일 (YYYY-MM-DD):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.start_date_var, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="종료일 (YYYY-MM-DD):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.end_date_var, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 2: 전용면적 (최소/최대)
        ttk.Label(parent, text="전용면적 (최소):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="전용면적 (최대):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 3: 연차, 평단가
        ttk.Label(parent, text="연차 (최대):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_years, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="최대 평단가:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_unit_price, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 4: 증감율 (최소/최대)
        ttk.Label(parent, text="증감율 (최소%):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="증감율 (최대%):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1

        # Row 5: 모든 버튼과 경로 UI 배치
        # 조회 버튼
        ttk.Button(parent, text="조회", command=self.search_data).grid(row=row, column=0, columnspan=2, pady=10)
    
        # Excel 내보내기 버튼
        ttk.Button(parent, text="Excel 내보내기", command=self.export_to_excel).grid(row=row, column=2, columnspan=2, pady=10)

        # 저장 경로 라벨
        ttk.Label(parent, text="저장 경로:", font=('Arial', 9)).grid(row=row, column=4, sticky=tk.W, padx=padx_val, pady=pady_val)
    
        # 저장 경로 표시 라벨
        self.path_display_label = ttk.Label(parent, textvariable=self.export_path_var, wraplength=180, justify=tk.LEFT, font=('Arial', 9))
        self.path_display_label.grid(row=row, column=5, columnspan=2, sticky="ew", padx=padx_val, pady=pady_val)
    
        # 경로 변경 버튼
        ttk.Button(parent, text="변경", command=self.choose_export_path, width=5).grid(row=row, column=7, sticky=tk.E, padx=padx_val, pady=pady_val)

    def on_sido_selected(self, event):
        """시도 선택 시 시군구 로드"""
        selected_sido = self.sido_var.get()
        if selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.sigungu_var.set(sigungu_list[0])

    def search_data(self):
        """투자테이블 데이터 검색 (임시 구현)"""
        messagebox.showinfo("정보", "투자테이블 데이터 검색 기능은 향후 구현 예정입니다.")

    def export_to_excel(self):
        """Excel 내보내기 (임시 구현)"""
        messagebox.showinfo("정보", "Excel 내보내기 기능은 향후 구현 예정입니다.")

    def choose_export_path(self):
        """저장 경로 선택"""
        path = tkinter.filedialog.askdirectory()
        if path:
            self.export_path_var.set(path)

    def create_result_table(self, parent):
        """투자테이블 결과 Treeview 생성"""
        table_frame = ttk.Frame(parent)
        table_frame.pack(fill=tk.BOTH, expand=True)

        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수')
        self.tree = ttk.Treeview(table_frame, columns=columns, show='tree headings', height=20)

        # 색상 태그 설정
        self.tree.tag_configure('positive_sale', background='#E8F5E8', foreground='#2E7D32')
        self.tree.tag_configure('negative_sale', background='#FFEBEE', foreground='#C62828')
        self.tree.tag_configure('positive_lease', background='#E3F2FD', foreground='#1565C0')
        self.tree.tag_configure('negative_lease', background='#FFF3E0', foreground='#E65100')
        self.tree.tag_configure('high_ratio', background='#FCE4EC', foreground='#AD1457')
        self.tree.tag_configure('oddrow', background='#F8F8F8')
        self.tree.tag_configure('evenrow', background='#FFFFFF')
        self.tree.tag_configure('neutral', background='#F5F5F5', foreground='#424242')

        self.tree.column("#0", width=30, minwidth=30, stretch=tk.NO)
        self.tree.heading("#0", text="", anchor=tk.CENTER)

        column_widths = {
            '지역': 120, '단지명': 150, '전용면적': 80, '연차': 60, '평단가': 100,
            '매매증감율': 100, '전세증감율': 100, '매매시세': 100, '전세시세': 100,
            '전세가율': 80, '매전갭': 80, '세대수': 80
        }

        self.sort_directions = {}

        # 헤더 스타일링
        style = ttk.Style()
        style.configure("Treeview.Heading", 
                       background="#2196F3",
                       foreground="black",
                       font=("Arial", 10, "bold"))
        style.map("Treeview.Heading",
                  background=[('active', '#1976D2')])

        for col in columns:
            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_column(_col))
            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
            self.sort_directions[col] = False

        scrollbar_y = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.tree.yview)
        scrollbar_x = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.tree.xview)

        self.tree.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

        self.tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

        self.tree.bind("<<TreeviewSelect>>", self._on_tree_select)

    def _sort_column(self, col):
        """테이블 컬럼 정렬 (임시 구현)"""
        pass

    def _on_tree_select(self, event):
        """테이블 선택 이벤트 (임시 구현)"""
        pass

    def create_chart_page(self, parent_frame):
        """2페이지: 시세차트 UI 생성 (매물 검색 기능 제거됨)"""
        pane_window = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        pane_window.pack(fill=tk.BOTH, expand=True)
        
        left_frame = ttk.Frame(pane_window, width=200, relief=tk.SUNKEN)
        pane_window.add(left_frame, weight=1)

        ttk.Label(left_frame, text="선택 단지 목록", font=('Arial', 14, 'bold')).pack(pady=10)

        selected_apt_columns = ('지역', '단지명', '전용면적', '연차')
        self.selected_apt_tree = ttk.Treeview(left_frame, columns=selected_apt_columns, show='headings', height=15)

        for col in selected_apt_columns:
            self.selected_apt_tree.heading(col, text=col, anchor=tk.CENTER)
            self.selected_apt_tree.column(col, width=80, anchor=tk.CENTER)

        selected_apt_scrollbar_y = ttk.Scrollbar(left_frame, orient=tk.VERTICAL, command=self.selected_apt_tree.yview)
        self.selected_apt_tree.configure(yscrollcommand=selected_apt_scrollbar_y.set)

        self.selected_apt_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        selected_apt_scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

        for item_id, apt_info in self.selected_apartments.items():
            self.selected_apt_tree.insert('', tk.END, values=(
                apt_info.get('지역', ''), apt_info.get('단지명', ''),
                apt_info.get('전용면적', ''), apt_info.get('연차', '')
            ), iid=item_id)

        self.selected_apt_tree.bind("<<TreeviewSelect>>", self.on_selected_apt_tree_select)

        # 중앙 프레임
        center_frame = ttk.Frame(pane_window, relief=tk.SUNKEN)
        pane_window.add(center_frame, weight=6)

        center_frame.grid_rowconfigure(0, weight=0)
        center_frame.grid_rowconfigure(1, weight=6)
        center_frame.grid_rowconfigure(2, weight=2)
        center_frame.grid_columnconfigure(0, weight=1)

        # 차트 옵션창
        chart_options_frame = ttk.LabelFrame(center_frame, text="차트 옵션", padding=10)
        chart_options_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

        # 차트 기간 설정 UI
        ttk.Label(chart_options_frame, text="차트 시작일:").grid(row=0, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_start_date_var, width=15).grid(row=0, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Label(chart_options_frame, text="차트 종료일:").grid(row=0, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_end_date_var, width=15).grid(row=0, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Button(chart_options_frame, text="기간 적용", command=self.update_chart_view).grid(row=0, column=4, padx=10, pady=2)

        # 차트 표시 유형 선택 체크버튼
        ttk.Checkbutton(chart_options_frame, text="매매가", variable=self.show_sise_sale_var, command=self.update_chart_view).grid(row=1, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가", variable=self.show_sise_lease_var, command=self.update_chart_view).grid(row=1, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가율", variable=self.show_sise_ratio_var, command=self.update_chart_view).grid(row=1, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="매매실거래", variable=self.show_real_sale_var, command=self.update_chart_view).grid(row=1, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세실거래", variable=self.show_real_lease_var, command=self.update_chart_view).grid(row=1, column=4, padx=5, pady=2, sticky=tk.W)

        # 시세차트 영역
        self.chart_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)

        self.fig = Figure(figsize=(6, 6), dpi=100)
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.chart_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # 차트 하단 주요 가격 정보
        self.chart_info_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_info_frame.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)

        # 3분할 프레임 및 레이블 생성
        price_info_frame1 = ttk.Frame(self.chart_info_frame)
        price_info_frame1.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        price_info_frame2 = ttk.Frame(self.chart_info_frame)
        price_info_frame2.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)
        price_info_frame3 = ttk.Frame(self.chart_info_frame)
        price_info_frame3.grid(row=0, column=2, sticky="nsew", padx=5, pady=5)
        
        self.chart_info_frame.grid_columnconfigure(0, weight=1)
        self.chart_info_frame.grid_columnconfigure(1, weight=1)
        self.chart_info_frame.grid_columnconfigure(2, weight=1)

        # 좌측 섹션: 매매 최고/최근가, 전세 최고/최근가
        ttk.Label(price_info_frame1, text="<시세 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels = {}
        self.chart_info_labels['매매최고가'] = ttk.Label(price_info_frame1, text="매매최고가: -")
        self.chart_info_labels['매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['매매최근가'] = ttk.Label(price_info_frame1, text="매매최근가: -")
        self.chart_info_labels['매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최고가'] = ttk.Label(price_info_frame1, text="전세최고가: -")
        self.chart_info_labels['전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최근가'] = ttk.Label(price_info_frame1, text="전세최근가: -")
        self.chart_info_labels['전세최근가'].pack(anchor=tk.W)

        # 가운데 섹션: 실거래 매매/전세 최고/최근가
        ttk.Label(price_info_frame2, text="<실거래 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['실거래매매최고가'] = ttk.Label(price_info_frame2, text="실거래매매최고가: -")
        self.chart_info_labels['실거래매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래매매최근가'] = ttk.Label(price_info_frame2, text="실거래매매최근가: -")
        self.chart_info_labels['실거래매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최고가'] = ttk.Label(price_info_frame2, text="실거래전세최고가: -")
        self.chart_info_labels['실거래전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최근가'] = ttk.Label(price_info_frame2, text="실거래전세최근가: -")
        self.chart_info_labels['실거래전세최근가'].pack(anchor=tk.W)

        # 우측 섹션: 매매 하락가/율, 회복율
        ttk.Label(price_info_frame3, text="<매매 분석>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매하락가'] = ttk.Label(price_info_frame3, text="매매하락가: -")
        self.chart_info_labels['매매하락가'].pack(anchor=tk.W)
        self.chart_info_labels['매매하락율'] = ttk.Label(price_info_frame3, text="매매하락율: -")
        self.chart_info_labels['매매하락율'].pack(anchor=tk.W)
        self.chart_info_labels['매매회복율'] = ttk.Label(price_info_frame3, text="매매회복율: -")
        self.chart_info_labels['매매회복율'].pack(anchor=tk.W)

        # 우측 프레임 (실거래 정보만 포함)
        right_frame = ttk.PanedWindow(pane_window, orient=tk.VERTICAL)
        pane_window.add(right_frame, weight=1)

        # 우측 상단 - 실거래 정보창 (좌우 2분할)
        top_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(top_right_frame, weight=1)

        top_right_frame.grid_columnconfigure(0, weight=1)
        top_right_frame.grid_columnconfigure(1, weight=1)
        top_right_frame.grid_rowconfigure(0, weight=0)
        top_right_frame.grid_rowconfigure(1, weight=1)

        # 매매 실거래 Treeview (좌측 컬럼)
        sale_label_frame = ttk.Frame(top_right_frame)
        sale_label_frame.grid(row=0, column=0, sticky="ew", padx=2, pady=2)
        ttk.Label(sale_label_frame, text="매매 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        sale_tree_frame = ttk.Frame(top_right_frame)
        sale_tree_frame.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        sale_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.sale_real_tran_tree = ttk.Treeview(sale_tree_frame, columns=sale_real_tran_cols, show='headings')
        for col in sale_real_tran_cols:
            self.sale_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.sale_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.sale_real_tran_tree.pack(fill=tk.BOTH, expand=True)

        # 전세/월세 실거래 Treeview (우측 컬럼)
        lease_label_frame = ttk.Frame(top_right_frame)
        lease_label_frame.grid(row=0, column=1, sticky="ew", padx=2, pady=2)
        ttk.Label(lease_label_frame, text="전세/월세 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        lease_tree_frame = ttk.Frame(top_right_frame)
        lease_tree_frame.grid(row=1, column=1, sticky="nsew", padx=2, pady=2)
        lease_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.lease_real_tran_tree = ttk.Treeview(lease_tree_frame, columns=lease_real_tran_cols, show='headings')
        for col in lease_real_tran_cols:
            self.lease_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.lease_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.lease_real_tran_tree.pack(fill=tk.BOTH, expand=True)

    def on_selected_apt_tree_select(self, event):
        """선택된 아파트 트리 선택 이벤트 (임시 구현)"""
        pass

    def update_chart_view(self):
        """차트 뷰 업데이트 (임시 구현)"""
        pass

    def create_timeseries_analysis_page(self, parent_frame):
        """3페이지: 시계열 분석 UI 생성"""
        # 메인 PanedWindow (3분할: 좌측, 중앙, 우측)
        main_paned = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        main_paned.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # === 좌측 영역 (다운로드 및 설정) ===
        left_frame = ttk.Frame(main_paned, width=300)
        main_paned.add(left_frame, weight=1)

        # 좌측 상단: 다운로드 버튼 및 경로 설정
        download_frame = ttk.LabelFrame(left_frame, text="시계열 다운로드", padding=10)
        download_frame.pack(fill=tk.X, padx=5, pady=5)

        # 다운로드 버튼들
        btn_frame = ttk.Frame(download_frame)
        btn_frame.pack(fill=tk.X, pady=(0, 10))

        kb_weekly_btn = ttk.Button(btn_frame, text="KB 주간시계열", 
                                  command=self.download_kb_weekly_timeseries)
        kb_weekly_btn.pack(side=tk.LEFT, padx=(0, 5), fill=tk.X, expand=True)

        kb_monthly_btn = ttk.Button(btn_frame, text="KB 월간시계열", 
                                   command=self.download_kb_monthly_timeseries)
        kb_monthly_btn.pack(side=tk.LEFT, padx=(5, 0), fill=tk.X, expand=True)

        # 다운로드 경로 설정
        path_frame = ttk.Frame(download_frame)
        path_frame.pack(fill=tk.X, pady=(0, 5))

        ttk.Label(path_frame, text="다운로드 경로:").pack(anchor=tk.W)
        
        path_entry_frame = ttk.Frame(path_frame)
        path_entry_frame.pack(fill=tk.X, pady=(5, 0))
        
        path_entry = ttk.Entry(path_entry_frame, textvariable=self.timeseries_download_path_var)
        path_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        path_browse_btn = ttk.Button(path_entry_frame, text="찾기", 
                                    command=self.browse_timeseries_download_path)
        path_browse_btn.pack(side=tk.RIGHT)

        # 좌측 하단: 분석 기준 선택
        analysis_frame = ttk.LabelFrame(left_frame, text="분석 설정", padding=10)
        analysis_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 시도/시군구 선택 (1행 2열)
        region_frame = ttk.Frame(analysis_frame)
        region_frame.pack(fill=tk.X, pady=(0, 10))

        # 1열: 시도 선택
        sido_frame = ttk.Frame(region_frame)
        sido_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        ttk.Label(sido_frame, text="시도:").pack(anchor=tk.W)
        self.timeseries_sido_combobox = ttk.Combobox(sido_frame, textvariable=self.timeseries_sido_var, 
                                                    state="readonly")
        self.timeseries_sido_combobox.pack(fill=tk.X, pady=(2, 0))
        self.timeseries_sido_combobox.bind("<<ComboboxSelected>>", self.on_timeseries_sido_selected)

        # 2열: 시군구 선택
        sigungu_frame = ttk.Frame(region_frame)
        sigungu_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(5, 0))
        ttk.Label(sigungu_frame, text="시군구:").pack(anchor=tk.W)
        self.timeseries_sigungu_combobox = ttk.Combobox(sigungu_frame, textvariable=self.timeseries_sigungu_var, 
                                                       state="readonly")
        self.timeseries_sigungu_combobox.pack(fill=tk.X, pady=(2, 0))

        # 3열: 기간 설정
        period_frame = ttk.Frame(analysis_frame)
        period_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Label(period_frame, text="분석 기간:").pack(anchor=tk.W)
        
        period_input_frame = ttk.Frame(period_frame)
        period_input_frame.pack(fill=tk.X, pady=(2, 0))

        ttk.Label(period_input_frame, text="시작:").pack(side=tk.LEFT)
        start_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_start_var, width=12)
        start_entry.pack(side=tk.LEFT, padx=(5, 10))

        ttk.Label(period_input_frame, text="종료:").pack(side=tk.LEFT)
        end_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_end_var, width=12)
        end_entry.pack(side=tk.LEFT, padx=(5, 0))

        # === 차트 데이터 선택 영역 ===
        chart_data_frame = ttk.LabelFrame(analysis_frame, text="차트 데이터 선택", padding=10)
        chart_data_frame.pack(fill=tk.X, pady=(10, 0))

        # 체크버튼들을 3x3 그리드로 배치
        chart_options = [
            ('매수심리', '매수심리'),
            ('전세현황', '전세현황'),
            ('전세수급', '전세수급'),
            ('매매전망', '매매전망'),
            ('전세전망', '전세전망'),
            ('매매지수', '매매지수'),
            ('전세지수', '전세지수'),
            ('매매증감', '매매증감'),
            ('전세증감', '전세증감')
        ]

        for i, (key, display_text) in enumerate(chart_options):
            row = i // 3
            col = i % 3
            cb = ttk.Checkbutton(chart_data_frame, text=display_text, 
                                variable=self.chart_data_vars[key],
                                command=self.on_chart_data_selection_changed)
            cb.grid(row=row, column=col, sticky=tk.W, padx=5, pady=2)

        # 전체 선택/해제 버튼
        select_all_frame = ttk.Frame(chart_data_frame)
        select_all_frame.grid(row=3, column=0, columnspan=3, pady=(10, 0))

        ttk.Button(select_all_frame, text="전체 선택", 
                  command=self.select_all_chart_data).pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(select_all_frame, text="전체 해제", 
                  command=self.deselect_all_chart_data).pack(side=tk.LEFT)

        # 분석 실행 버튼
        analyze_btn = ttk.Button(analysis_frame, text="차트 생성", 
                               command=self.generate_timeseries_chart)
        analyze_btn.pack(fill=tk.X, pady=(10, 0))

        # === 우측 영역 (차트 표시) ===
        right_frame = ttk.Frame(main_paned)
        main_paned.add(right_frame, weight=2)

        # 차트 프레임
        self.timeseries_chart_frame = ttk.LabelFrame(right_frame, text="시계열 분석 차트", padding=10)
        self.timeseries_chart_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 초기 차트 설정
        self.setup_timeseries_chart()

        # 시도/시군구 콤보박스 초기 데이터 로드
        self.load_timeseries_region_data()

    def browse_timeseries_download_path(self):
        """시계열 다운로드 경로 선택"""
        path = tkinter.filedialog.askdirectory()
        if path:
            self.timeseries_download_path_var.set(path)

    def on_timeseries_sido_selected(self, event):
        """시계열 시도 선택 이벤트 (임시 구현)"""
        pass

    def on_chart_data_selection_changed(self):
        """차트 데이터 선택 변경 이벤트 (임시 구현)"""
        pass

    def select_all_chart_data(self):
        """모든 차트 데이터 선택"""
        for var in self.chart_data_vars.values():
            var.set(True)

    def deselect_all_chart_data(self):
        """모든 차트 데이터 선택 해제"""
        for var in self.chart_data_vars.values():
            var.set(False)

    def generate_timeseries_chart(self):
        """시계열 차트 생성 (임시 구현)"""
        messagebox.showinfo("정보", "시계열 차트 생성 기능은 향후 구현 예정입니다.")

    def setup_timeseries_chart(self):
        """시계열 차트 초기 설정"""
        self.timeseries_fig = Figure(figsize=(8, 6), dpi=100)
        self.timeseries_ax = self.timeseries_fig.add_subplot(111)
        self.timeseries_canvas = FigureCanvasTkAgg(self.timeseries_fig, master=self.timeseries_chart_frame)
        self.timeseries_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def load_timeseries_region_data(self):
        """시계열 분석용 지역 데이터 로드"""
        if self.region_data:
            self.timeseries_sido_combobox['values'] = list(self.region_data.keys())

    def create_property_collector_page(self, parent_frame):
        """4페이지: 매물수집기 UI 생성"""
        # 입력 프레임 구성
        input_frame = tk.Frame(parent_frame, padx=10, pady=10, relief="groove", bd=2)
        input_frame.pack(pady=10, fill="x")

        # ----- 제1세로열: 지역 선택 콤보박스 (Column 0-1) -----
        combobox_label_col = 0 
        combobox_widget_col = 1 

        current_row_for_combobox = 0 
        tk.Label(input_frame, text="시/도 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_sido_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_sido_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_sido_combobox.bind("<<ComboboxSelected>>", self.on_collector_sido_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="시/군/구 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_gungu_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_gungu_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_gungu_combobox.bind("<<ComboboxSelected>>", self.on_collector_gungu_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="법정동 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_legal_dong_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_legal_dong_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_legal_dong_combobox.bind("<<ComboboxSelected>>", self.on_collector_legal_dong_selected)

        # 법정동 코드 저장을 위한 StringVar
        self.cortar_no_var = tk.StringVar() 
        # 좌표 정보 저장용
        self.coord_vars = {
            'lat': tk.StringVar(), 'lon': tk.StringVar(), 'btm': tk.StringVar(),
            'lft': tk.StringVar(), 'top': tk.StringVar(), 'rgt': tk.StringVar()
        }

        # ----- 중앙 영역: 체크박스 그룹들 (Column 2-7) -----
        checkbox_label_start_col = 2 
        checkbox_widget_start_col = 3 

        checkbox_current_row = 0 

        # --- 부동산 유형 (체크박스 그룹) ---
        tk.Label(input_frame, text="부동산 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        rlet_type_checkbox_frame = tk.Frame(input_frame)
        rlet_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.rlet_type_vars = {} 
        for name in self._RLET_TYPE_CODES.keys():
            var = tk.BooleanVar(value=(name == '아파트')) 
            cb = tk.Checkbutton(rlet_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.rlet_type_vars[name] = var

        # --- 거래 유형 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="거래 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        trad_type_checkbox_frame = tk.Frame(input_frame)
        trad_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.trad_type_vars = {} 
        for name in self._TRAD_TYPE_CODES.keys():
            var = tk.BooleanVar(value=True) 
            cb = tk.Checkbutton(trad_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.trad_type_vars[name] = var
        
        # --- 평형대 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="평형대:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        pyeong_type_checkbox_frame = tk.Frame(input_frame)
        pyeong_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w", rowspan=2)
        self.pyeong_type_vars = {} 
        
        # 평형대 체크박스를 2행으로 나누어 나열
        pyeong_names = list(self._PYEONG_TYPE_RANGES.keys())
        num_cols_per_row_in_frame = (len(pyeong_names) + 1) // 2 
        
        for i, name in enumerate(pyeong_names):
            row_in_frame = i // num_cols_per_row_in_frame 
            col_in_frame = i % num_cols_per_row_in_frame 
            
            var = tk.BooleanVar(value=False) 
            cb = tk.Checkbutton(pyeong_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.grid(row=row_in_frame, column=col_in_frame, padx=5, sticky="w") 
            self.pyeong_type_vars[name] = var

        # ----- 오른쪽 영역: 설정 및 버튼들 (Column 8-9) -----
        right_section_label_col = 8 
        right_section_widget_col = 9 

        # 줌 레벨 설정 (숨김)
        self.z_level_var = tk.IntVar(value=12)
        # 최대 페이지 설정 (숨김)
        self.max_pages_var = tk.IntVar(value=50)
        
        # --- 버튼 재배치 ---
        # 매물 조회 버튼
        query_button_row = current_row_for_combobox + 1 
        self.collector_query_button = tk.Button(input_frame, text="매물 조회", command=self.start_collector_fetch_thread, bg="#007BFF", fg="white", font=('맑은 고딕', 10, 'bold'), width=10) 
        self.collector_query_button.grid(row=query_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 

        # 중지 버튼
        stop_button_row = query_button_row + 1 
        self.collector_stop_button = tk.Button(input_frame, text="중지", command=self.stop_collector_fetch, bg="#FF4500", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED, width=10) 
        self.collector_stop_button.grid(row=stop_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 

        # 저장 공간 입력창 및 버튼
        save_section_start_row = 0 
        tk.Label(input_frame, text="저장 경로:", font=('맑은 고딕', 10, 'bold')).grid(row=save_section_start_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.collector_save_path_var = tk.StringVar(value="results.csv") 
        tk.Entry(input_frame, textvariable=self.collector_save_path_var, width=20, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        
        save_section_start_row += 1
        tk.Button(input_frame, text="저장 경로 선택", command=self.select_collector_save_path, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=5, sticky="ew")

        save_section_start_row += 1
        self.collector_save_button = tk.Button(input_frame, text="조회 결과 저장", command=self.save_collector_articles_to_file, bg="#28A745", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED)
        self.collector_save_button.grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=10, sticky="ew")

        # 그리드 컬럼 설정
        input_frame.grid_columnconfigure(combobox_widget_col, weight=1) 
        input_frame.grid_columnconfigure(checkbox_widget_start_col, weight=1) 
        input_frame.grid_columnconfigure(right_section_widget_col, weight=1) 

        # 결과용 테이블
        table_frame = tk.Frame(parent_frame, padx=10, pady=10)
        table_frame.pack(fill="both", expand=True)

        columns = ("atclNo", "atclNm", "tradTpNm", "hanPrc", "prc", "rentPrc", "flrInfo", "spc1", "spc2", "direction", "cortarNo")
        self.collector_tree = ttk.Treeview(table_frame, columns=columns, show="headings")

        headers = ["매물번호", "매물명", "거래종류", "한글가격", "매매/전세가", "월세가",
                   "층정보", "공급(㎡)", "전용(㎡)", "방향", "법정동코드"]

        for col, text in zip(columns, headers):
            self.collector_tree.heading(col, text=text)
            self.collector_tree.column(col, width=100, anchor="center")

        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=self.collector_tree.yview)
        vsb.pack(side="right", fill="y")
        self.collector_tree.configure(yscrollcommand=vsb.set)

        hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=self.collector_tree.xview)
        hsb.pack(side="bottom", fill="x")
        self.collector_tree.configure(xscrollcommand=hsb.set)

        self.collector_tree.pack(fill="both", expand=True)

        self.status_label = tk.Label(parent_frame, text="", bd=1, relief="sunken", anchor="w")
        self.status_label.pack(side="bottom", fill="x")

        # 지역 데이터 저장용
        self._collector_regions_data = {
            'sido': {},
            'gungu': {},
            'legal_dong': {}
        }

        # 앱 시작 후 시도 목록 자동 로드
        self.root.after(100, self.load_collector_sido_list) 

    # 매물수집기 관련 메서드들
    
    def load_collector_sido_list(self): 
        """네이버 부동산 시도 목록 로드"""
        self.update_status("시/도 목록 로드 중...")
        
        url = GET_REGION_LIST_API_URL 
        params = {"cortarNo": "0000000000", "mycortarNo": ""}
        
        data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 시도") 
        
        if data and 'result' in data and 'list' in data['result']:
            sido_list_raw = data['result']['list']
            
            sido_names = []
            self._collector_regions_data['sido'] = {} 
            for item in sido_list_raw:
                sido_name = item.get('CortarNm')
                sido_code = item.get('CortarNo')
                if sido_name and sido_code:
                    sido_names.append(sido_name)
                    self._collector_regions_data['sido'][sido_name] = sido_code
            
            self.collector_sido_combobox['values'] = sido_names
            
            self.collector_sido_combobox.unbind("<<ComboboxSelected>>")
            if sido_names:
                self.collector_sido_combobox.set(sido_names[0]) 
                self.on_collector_sido_selected(None)
            else:
                self.collector_sido_combobox.set("시도 선택")
            self.collector_sido_combobox.bind("<<ComboboxSelected>>", self.on_collector_sido_selected)
            self.update_status("시/도 목록 로드 완료.")
        else:
            self.update_status("⛔️ 시/도 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", "시도 목록을 가져오지 못했습니다.")) 
            self.collector_sido_combobox['values'] = ["목록 로드 실패"]
            self.collector_sido_combobox.set("목록 로드 실패")
        self.root.update_idletasks()

    def on_collector_sido_selected(self, event):
        selected_sido_name = self.collector_sido_combobox.get()
        selected_sido_code = self._collector_regions_data['sido'].get(selected_sido_name)
        
        if not selected_sido_code:
            return

        self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 중...")
        self.collector_gungu_combobox.set('')
        self.collector_gungu_combobox['values'] = []
        self.collector_legal_dong_combobox.set('')
        self.collector_legal_dong_combobox['values'] = []
        self._collector_regions_data['gungu'] = {} 
        self._collector_regions_data['legal_dong'] = {} 

        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_sido_code, "mycortarNo": ""}
        
        response_data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 시군구")
        
        if response_data and 'result' in response_data and 'list' in response_data['result']:
            gungu_list_raw = response_data['result']['list']
            gungu_names = []
            for gungu_data in gungu_list_raw:
                name = gungu_data.get('CortarNm')
                code = gungu_data.get('CortarNo')
                if name and code:
                    gungu_names.append(name)
                    self._collector_regions_data['gungu'][name] = {'code': code}
            self.collector_gungu_combobox['values'] = gungu_names
            self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_sido_name}'의 시군구 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", f"'{selected_sido_name}' 시군구 목록을 가져오지 못했습니다."))

    def on_collector_gungu_selected(self, event):
        selected_gungu_name = self.collector_gungu_combobox.get()
        gungu_info = self._collector_regions_data['gungu'].get(selected_gungu_name)
        
        if not gungu_info:
            return

        selected_gungu_code = gungu_info.get('code')

        self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 중...")
        self.collector_legal_dong_combobox.set('')
        self.collector_legal_dong_combobox['values'] = []
        self._collector_regions_data['legal_dong'] = {}

        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_gungu_code, "mycortarNo": ""}

        response_data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 법정동")

        if response_data and 'result' in response_data and 'list' in response_data['result']:
            legal_dong_list_raw = response_data['result']['list']
            legal_dong_names = []
            for legal_dong_data in legal_dong_list_raw:
                name = legal_dong_data.get('CortarNm')
                code = legal_dong_data.get('CortarNo')
                if name and code:
                    legal_dong_names.append(name)
                    self._collector_regions_data['legal_dong'][name] = {
                        'code': code,
                        'lat_center': float(legal_dong_data.get('MapYCrdn')),
                        'lon_center': float(legal_dong_data.get('MapXCrdn')),
                    }
            self.collector_legal_dong_combobox['values'] = legal_dong_names
            self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_gungu_name}'의 법정동 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", f"'{selected_gungu_name}' 법정동 목록을 가져오지 못했습니다."))

    def on_collector_legal_dong_selected(self, event):
        selected_legal_dong_name = self.collector_legal_dong_combobox.get()
        legal_dong_info = self._collector_regions_data['legal_dong'].get(selected_legal_dong_name)

        if not legal_dong_info:
            return
        
        self.cortar_no_var.set(legal_dong_info.get('code', '')) 
        
        center_lat = legal_dong_info.get('lat_center')
        center_lon = legal_dong_info.get('lon_center')

        if center_lat is not None and center_lon is not None:
            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14

            deltas = self._Z_LEVEL_DELTAS.get(z_level, self._Z_LEVEL_DELTAS[14]) 
            lat_delta = deltas['lat_delta']
            lon_delta = deltas['lon_delta']

            btm_val = center_lat - lat_delta
            top_val = center_lat + lat_delta
            lft_val = center_lon - lon_delta
            rgt_val = center_lon + lon_delta
            
            self.coord_vars['lat'].set(str(center_lat))
            self.coord_vars['lon'].set(str(center_lon))
            self.coord_vars['btm'].set(str(btm_val))
            self.coord_vars['lft'].set(str(lft_val))
            self.coord_vars['top'].set(str(top_val))
            self.coord_vars['rgt'].set(str(rgt_val))
            
            self.update_status(f"'{selected_legal_dong_name}' 선택 완료. 좌표 자동 입력됨.")
        else:
            for var in self.coord_vars.values():
                var.set('')
            self.update_status(f"'{selected_legal_dong_name}' 선택 완료. 중심 좌표 정보가 없어 자동 입력 불가.")

    def start_collector_fetch_thread(self):
        if hasattr(self, 'collector_fetch_thread') and self.collector_fetch_thread.is_alive():
            self.root.after(0, lambda: messagebox.showwarning("경고", "이미 매물 조회 작업이 진행 중입니다."))
            return
        self._stop_flag = False
        self.root.after(0, lambda: self.collector_query_button.config(state=tk.DISABLED))
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.NORMAL))
        self.collector_save_button.config(state=tk.DISABLED)
        self.update_status("매물 조회 시작...")
        self.fetched_article_data = []

        self.collector_fetch_thread = threading.Thread(target=self.fetch_collector_articles, daemon=True)
        self.collector_fetch_thread.start()

    def stop_collector_fetch(self):
        self._stop_flag = True
        self.update_status("중지 요청됨. 현재 페이지 작업 완료 후 중단됩니다.")
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.DISABLED))

    def fetch_collector_articles(self):
        cortar_no = self.cortar_no_var.get().strip() 
        max_pages = self.max_pages_var.get()

        lat = self.coord_vars['lat'].get().strip()
        lon = self.coord_vars['lon'].get().strip()
        btm = self.coord_vars['btm'].get().strip()
        lft = self.coord_vars['lft'].get().strip()
        top = self.coord_vars['top'].get().strip()
        rgt = self.coord_vars['rgt'].get().strip()

        # 부동산 유형 체크박스 상태를 기반으로 최종 rletTpCd 생성
        selected_rlet_types = []
        for name, var in self.rlet_type_vars.items():
            if var.get():
                api_code_str = self._RLET_TYPE_CODES.get(name, '') 
                if api_code_str:
                    selected_rlet_types.extend(api_code_str.split(':'))
        
        rlet_tp_cd_final = ":".join(sorted(list(set(selected_rlet_types))))

        # 거래 유형 체크박스 상태를 기반으로 최종 tradTpCd 생성
        selected_trad_types = []
        for name, var in self.trad_type_vars.items():
            if var.get(): 
                api_code = self._TRAD_TYPE_CODES.get(name, '')
                if api_code:
                    selected_trad_types.append(api_code)
        
        trad_tp_cd_final = ":".join(sorted(list(set(selected_trad_types))))

        # 평형대 체크박스 상태를 기반으로 spcMin/spcMax 생성
        min_spc = float('inf')  
        max_spc = 0.0

        is_pyeong_selected = False
        for name, var in self.pyeong_type_vars.items():
            if var.get():
                is_pyeong_selected = True
                pyeong_range = self._PYEONG_TYPE_RANGES[name]
                min_val = pyeong_range.get('spcMin', 0) 
                max_val = pyeong_range.get('spcMax', float('inf')) 

                min_spc = min(min_spc, min_val)
                max_spc = max(max_spc, max_val)
        
        params_spcMin = None
        params_spcMax = None
        if is_pyeong_selected:
            params_spcMin = str(int(min_spc)) if min_spc != float('inf') else None 
            params_spcMax = str(int(max_spc)) if max_spc != float('inf') else None 

        if not cortar_no:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "법정동을 선택하세요."))
            self.reset_collector_buttons_state()
            return
        if max_pages <= 0:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "최대 페이지 수는 1 이상이어야 합니다."))
            self.reset_collector_buttons_state()
            return
        
        if not rlet_tp_cd_final: 
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 부동산 유형을 선택해주세요."))
            self.reset_collector_buttons_state()
            return

        if not trad_tp_cd_final: 
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 거래 유형을 선택해주세요."))
            self.reset_collector_buttons_state()
            return

        if not is_pyeong_selected: 
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 평형대를 선택해주세요."))
            self.reset_collector_buttons_state()
            return
        
        if not (lat and lon and btm and lft and top and rgt): 
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "선택된 법정동의 지도 좌표 정보가 불완전합니다."))
            self.reset_collector_buttons_state()
            return

        self.root.after(0, lambda: [self.collector_tree.delete(i) for i in self.collector_tree.get_children()])
        total_count = 0
        self.fetched_article_data = []
        self.update_status("매물 조회 중...")

        for page in range(1, max_pages + 1):
            if self._stop_flag:
                self.update_status("사용자 요청으로 조회 중단됨.")
                break

            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14

            params = {
                "itemId": "", "mapKey": "", "lgeo": "", "showR0": "",
                "rletTpCd": rlet_tp_cd_final, 
                "tradTpCd": trad_tp_cd_final, 
                "z": str(z_level),
                "lat": lat, "lon": lon, "btm": btm, "lft": lft, "top": top, "rgt": rgt,
                "totCnt": "0",
                "cortarNo": cortar_no,
                "sort": "rank",
                "page": page
            }
            if params_spcMin is not None:
                params['spcMin'] = params_spcMin
            if params_spcMax is not None:
                params['spcMax'] = params_spcMax
            
            response = self._safe_request('get', GET_API_URL, NAVER_HEADERS, params=params, api_name="매물 조회 API")
            if response == "RETRY":
                page -= 1
                continue
            
            if not response or 'body' not in response: 
                self.update_status("매물 조회 실패 또는 매물 없음.")
                break

            articles = response['body']
            if not articles:
                self.update_status("더 이상 매물이 존재하지 않습니다.")
                break

            for art in articles:
                # Treeview에 표시할 데이터
                tree_values = (
                    art.get('atclNo', 'N/A'),
                    art.get('atclNm', 'N/A'),
                    art.get('tradTpNm', 'N/A'),
                    art.get('hanPrc', 'N/A'),
                    art.get('prc', 'N/A'),
                    art.get('rentPrc', 'N/A'),
                    art.get('flrInfo', 'N/A'),
                    art.get('spc1', 'N/A'),
                    art.get('spc2', 'N/A'),
                    art.get('direction', 'N/A'),
                    art.get('cortarNo', 'N/A')
                )
                self.root.after(0, lambda a=tree_values: self.collector_tree.insert("", "end", values=a))

                # pandas 저장을 위한 원본 데이터를 저장
                self.fetched_article_data.append(art)

            total_count += len(articles)

            if page < max_pages:
                delay = random.uniform(0, 1) 
                self.update_status(f"{page}페이지 완료. 다음 페이지 조회 전 {delay:.2f}초 대기 중...")
                time.sleep(delay)

        self.update_status(f"총 {total_count}개의 매물을 테이블에 표시했습니다.")
        self.reset_collector_buttons_state()
        self.root.after(0, lambda: messagebox.showinfo("완료", f"총 {total_count}개의 매물을 가져왔습니다."))
        if self.fetched_article_data:
            self.root.after(0, lambda: self.collector_save_button.config(state=tk.NORMAL))

    def reset_collector_buttons_state(self):
        self.root.after(0, lambda: self.collector_query_button.config(state=tk.NORMAL))
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.DISABLED))

    def select_collector_save_path(self):
        selected_dong_name = self.collector_legal_dong_combobox.get()
        if not selected_dong_name:
            selected_dong_name = "매물"
        
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        default_filename = f"{selected_dong_name}_{timestamp}.csv"

        file_path = tkinter.filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
            initialfile=default_filename
        )
        if file_path:
            self.collector_save_path_var.set(file_path)

    def save_collector_articles_to_file(self):
        if not self.fetched_article_data:
            messagebox.showwarning("저장 오류", "저장할 매물 데이터가 없습니다.")
            return
        
        file_path = self.collector_save_path_var.get()
        if not file_path:
            messagebox.showwarning("저장 오류", "저장 경로를 입력해주세요.")
            return

        try:
            df = pd.DataFrame(self.fetched_article_data)
            
            # 매핑 정의
            column_mapping = {
                'atclNo': "매물번호", 'atclNm': "매물명", 'tradTpNm': "거래종류", 
                'hanPrc': "한글가격", 'prc': "매매/전세가", 'rentPrc': "월세가", 
                'flrInfo': "층정보", 'spc1': "공급(㎡)", 'spc2': "전용(㎡)", 
                'direction': "방향", 'cortarNo': "법정동코드"
            }
            # DataFrame에서 필요한 컬럼만 추출하고 순서 정렬
            cols_to_extract = list(column_mapping.keys())
            df_filtered = df[cols_to_extract].copy()
            # 컬럼명 변경
            df_filtered = df_filtered.rename(columns=column_mapping)
            
            # CSV로 저장 (UTF-8 인코딩)
            df_filtered.to_csv(file_path, index=False, encoding='utf-8-sig') 
            messagebox.showinfo("저장 완료", f"매물 데이터가 '{file_path}'에 성공적으로 저장되었습니다.")
            self.update_status(f"데이터를 '{file_path}'에 저장했습니다.")
        except Exception as e:
            messagebox.showerror("저장 오류", f"파일 저장 중 오류가 발생했습니다: {e}")
            self.update_status(f"⛔️ 저장 오류: {e}")

    # 프로그램 실행 함수
    def run(self):
        self.root.mainloop()

# 메인 실행부
if __name__ == "__main__":
    current_date = datetime.date.today()

    # 프로그램 만료일 확인
    if current_date > EXPIRATION_DATE:
        messagebox.showerror("프로그램 만료", f"프로그램 사용 기한이 {EXPIRATION_DATE}로 만료되었습니다.\n더 이상 프로그램을 사용할 수 없습니다.")
        import sys
        sys.exit()
    
    # 만료 예정일 알림 확인
    expiration_warning_date = EXPIRATION_DATE - datetime.timedelta(days=30) 

    if current_date >= expiration_warning_date and current_date <= EXPIRATION_DATE:
        remaining_days = (EXPIRATION_DATE - current_date).days
        messagebox.showwarning(
            "사용 기한 임박", 
            f"⚠️ 프로그램 사용 기한이 {EXPIRATION_DATE}까지입니다. (남은 기간: {remaining_days}일)\n"
            "미리 준비해 주세요."
        )

    # 프로그램 실행
    app = InvestmentTableProgram(PROGRAM_EXPIRATION_DATE_STR)
    app.run()

pywin32 라이브러리를 성공적으로 로드했습니다.
DPI 인식 설정을 시도했습니다.


AttributeError: module 'datetime' has no attribute 'strptime'